In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [8]:
device = torch.device("cpu")

In [10]:
import pandas as pd

# 거래 내역 엑셀파일 읽기
df = pd.read_excel("./transaction.xls")

# 거래일이 없는 행 제거 (null 또는 빈 문자열)
df = df[df["거래일"].notna() & (df["거래일"].str.strip() != "")]

# JSON 파일로 저장
df.to_json("output.json", orient="records", force_ascii=False, indent=2)

In [12]:
from dotenv import load_dotenv
import openai
from openai import OpenAI
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client=OpenAI(api_key=api_key)

In [14]:
from datasets import load_dataset

# JSON 데이터 로드
dataset = load_dataset("json", data_files="output.json")["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
print(dataset)

Dataset({
    features: ['거래일', '카드구분', '이용카드', '가맹점명', '금액', '이용구분', '거래통화', '해외이용금액', '취소상태'],
    num_rows: 143
})


In [18]:
store_names = dataset["가맹점명"]
unique_stores = list(set(store_names))
unique_stores

['서초제일마디의원',
 '홍등',
 '옴므',
 '(주)비바리퍼블리카',
 '주식회사 함흥면옥',
 '긴자료코 서초점',
 '남도식당',
 'LG  U+통신요금  자동이체',
 '하루',
 '부탄츄',
 '핵밥 건대점',
 '주식회사 유피소프트',
 '왓더버거 서초점',
 '홈플러스',
 'Amazon_AWS',
 '메가엠지씨커피 신대방역점',
 'Apple',
 '씨알지(c.r.g)-신림점',
 'OPENAI *CHATGPT SUBSCR',
 '( 주)버거요강남본점',
 '봉산옥',
 '쿠팡이츠',
 '(주)두잇',
 '고벤트 주식회사',
 '여기어때',
 '포마토김밥',
 '송황 칼국수',
 '탐앤탐스 광진화양점',
 '런드리킹7호점',
 '지금연구소 나우카페',
 '서초백화점약국',
 '서울옥',
 '서울도시가스(주)',
 '우먼센스',
 '메가엠지씨커피 건대스타점',
 '진흥마트',
 '주식회사 놀유니버스',
 '초라멘',
 '티머니 개인택시',
 '나오리주물럭 남부터미널점',
 '현대푸드(또봉이통닭&별난만두)',
 '세븐스타 코인노래연습장',
 '(주)에프이지',
 '(주)팀플러스',
 '(주)어비즈',
 '(주)갈라인터내셔널',
 '길동우동 건대점',
 '고향촌',
 '곱창의전당 서초본점',
 'CU 건대상허도서관점',
 '주식회사 리베르 관악 1지점',
 '파덕 오리바베큐',
 '한설원',
 '지에스GS25 서초한신점',
 '쇼군',
 '에이스아메리카화재해상보험',
 '써브웨이건대입구',
 'GS25신림수성점',
 '벨렘351 건대점',
 '동래정 선릉직영점',
 '(주)아트박스 건대스타시티점',
 '나카노라멘',
 '카츠공방',
 '홍루몽',
 '김가네김밥 건대점',
 'AXA손해보험',
 '쿠팡',
 '서초칼국수 닭한마리',
 '영길리',
 '(주)신세계프라퍼티 코엑스몰',
 '컴포즈커피 테헤란아이파크점',
 '롯데시네마 신림 (온라인 티켓)',
 '씨유 관악건영점',
 'OPENAI',
 '세븐스애비뉴 (7th Avenue)',
 '건대36

In [20]:
def classify_store(store):
    prompt = f"""
    가맹점명: {store}
    이 가맹점의 업종을 다음 중 하나로 분류하세요: [음식, 카페, 쇼핑, 디지털, 교통, 의료, 보험, 통신, 공과금, 생활용품, 오락, 기타]

    결과는 업종명만 한 단어로 출력하세요.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return "기타"

In [22]:
results = []
for store in unique_stores:
    category = classify_store(store)
    print(f"{store} → {category}")
    results.append({"store": store, "category": category})

서초제일마디의원 → 의료
홍등 → 음식
옴므 → 쇼핑
(주)비바리퍼블리카 → 디지털
주식회사 함흥면옥 → 음식
긴자료코 서초점 → 음식
남도식당 → 음식
LG  U+통신요금  자동이체 → 통신
하루 → 음식
부탄츄 → 음식
핵밥 건대점 → 음식
주식회사 유피소프트 → 디지털
왓더버거 서초점 → 음식
홈플러스 → 쇼핑
Amazon_AWS → 디지털
메가엠지씨커피 신대방역점 → 카페
Apple → 디지털
씨알지(c.r.g)-신림점 → 기타
OPENAI *CHATGPT SUBSCR → 디지털
( 주)버거요강남본점 → 음식
봉산옥 → 음식
쿠팡이츠 → 음식
(주)두잇 → 기타
고벤트 주식회사 → 기타
여기어때 → 기타
포마토김밥 → 음식
송황 칼국수 → 음식
탐앤탐스 광진화양점 → 카페
런드리킹7호점 → 기타
지금연구소 나우카페 → 카페
서초백화점약국 → 의료
서울옥 → 음식
서울도시가스(주) → 기타
우먼센스 → 쇼핑
메가엠지씨커피 건대스타점 → 카페
진흥마트 → 쇼핑
주식회사 놀유니버스 → 오락
초라멘 → 음식
티머니 개인택시 → 교통
나오리주물럭 남부터미널점 → 음식
현대푸드(또봉이통닭&별난만두) → 음식
세븐스타 코인노래연습장 → 오락
(주)에프이지 → 기타
(주)팀플러스 → 기타
(주)어비즈 → 기타
(주)갈라인터내셔널 → 기타
길동우동 건대점 → 음식
고향촌 → 음식
곱창의전당 서초본점 → 음식
CU 건대상허도서관점 → 기타
주식회사 리베르 관악 1지점 → 기타
파덕 오리바베큐 → 음식
한설원 → 음식
지에스GS25 서초한신점 → 쇼핑
쇼군 → 음식
에이스아메리카화재해상보험 → 보험
써브웨이건대입구 → 음식
GS25신림수성점 → 쇼핑
벨렘351 건대점 → 음식
동래정 선릉직영점 → 음식
(주)아트박스 건대스타시티점 → 쇼핑
나카노라멘 → 음식
카츠공방 → 음식
홍루몽 → 음식
김가네김밥 건대점 → 음식
AXA손해보험 → 보험
쿠팡 → 쇼핑
서초칼국수 닭한마리 → 음식
영길리 → 음식
(주)신세계프라퍼티 코엑스몰 → 쇼핑
컴포즈커피 테헤란아이파크점 → 카페


In [23]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

embeddings = OpenAIEmbeddings()

# Document 생성
docs = [
    Document(page_content=store["store"], metadata={"category": store["category"]})
    for store in results
]

# 벡터 DB 생성
vectorstore = FAISS.from_documents(docs, embedding=embeddings)

# 저장
vectorstore.save_local("./vector_db/stores")

/var/folders/yz/dp_hy1vd5jj8pzrlts8w9z4m0000gn/T/ipykernel_80102/3666435399.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [24]:
# 저장된 DB 확인
os.listdir("./vector_db/stores")

['index.faiss', 'index.pkl']

In [ ]:
# 벡터 DB 로드
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

vectorstore = FAISS.load_local(
    "./vector_db/stores",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True
)

# 모델 및 토크나이저
tokenizer = AutoTokenizer.from_pretrained("PleIAs/Pleias-RAG-1B")
model = AutoModelForCausalLM.from_pretrained("PleIAs/Pleias-RAG-1B").to(device)

# padding 토큰 아예 사용하지 않도록 처리
tokenizer.pad_token = tokenizer.eos_token  # 혹은 skip

# 유사한 문맥 가져오기
def get_similar_context(store_name: str, k=3, max_chars=500):
    docs = vectorstore.similarity_search(store_name, k=k)
    return "\n".join([f"- {doc.page_content}" for doc in docs])[:max_chars]

# 추론
def rag_infer(store_name: str, context: str):
    prompt = f"""[QUESTION]
    "{store_name}"는 어떤 업종인가요?

    [CONTEXT]
    {context}

    [ANSWER]"""

    # padding 없이 encoding (truncation만 사용)
    encoded = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    encoded = {k: v.to(device) for k, v in encoded.items()}

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=encoded["input_ids"],
            attention_mask=encoded["attention_mask"],
            max_new_tokens=16,
            do_sample=False,
            temperature=0.0,
        )

    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return decoded.split("[ANSWER]")[-1].strip()

# 결과 갱신
for entry in results:
    if entry["category"] == "기타":
        print(f"\n기타 재분류 시도: {entry['store']}")
        context = get_similar_context(entry["store"], k=3)
        if not context.strip():
            print("context 없음 → 기타 유지")
            continue
        new_category = rag_infer(entry["store"], context)
        print(f"✅ {entry['store']} → {new_category}")
        entry["category"] = new_category


기타 재분류 시도: (주)어비즈


In [28]:
# 최신 results를 기반으로 문서 재생성
updated_docs = [
    Document(page_content=entry["store"], metadata={"category": entry["category"]})
    for entry in results
]

# 임베딩 모델
embedding_model = OpenAIEmbeddings()

# 벡터스토어 재생성
updated_vectorstore = FAISS.from_documents(updated_docs, embedding=embedding_model)

# 기존 경로에 덮어쓰기 저장
updated_vectorstore.save_local("./vector_db/stores")

In [34]:
import json

corpus = [
    {
        "instruction": "다음 가맹점의 업종을 분류하세요.",
        "input": entry["store"],
        "output": entry["category"]
    }
    for entry in results
]

with open("./corpus.json", "w", encoding="utf-8") as f:
    json.dump(corpus, f, ensure_ascii=False, indent=2)

In [36]:
corpus

[{'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '서초제일마디의원', 'output': '의료'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '홍등', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '옴므', 'output': '쇼핑'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '(주)비바리퍼블리카', 'output': '디지털'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '주식회사 함흥면옥', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '긴자료코 서초점', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '남도식당', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': 'LG  U+통신요금  자동이체',
  'output': '통신'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '하루', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '부탄츄', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '핵밥 건대점', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '주식회사 유피소프트', 'output': '디지털'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '왓더버거 서초점', 'output': '음식'},
 {'instru

In [38]:
import random

# shuffle한 뒤 분할
random.shuffle(corpus)
split_idx = int(len(corpus) * 0.8)
train_data = corpus[:split_idx]
validation_data = corpus[split_idx:]

In [40]:
train_data

[{'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '벨렘351 건대점', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '홍등', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '쿠팡', 'output': '쇼핑'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '송황 칼국수', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '주식회사 놀유니버스', 'output': '오락'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '씨알지(c.r.g)-신림점',
  'output': '기타'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '영길리', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '고향촌', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '봉산옥', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '김가네김밥 건대점', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '(주)카카오', 'output': '디지털'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '(주)신세계프라퍼티 코엑스몰',
  'output': '쇼핑'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '쇼군', 'output': '음식'},
 {'instruc

In [44]:
validation_data

[{'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '한설원', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '서울옥', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '서초칼국수 닭한마리', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '써브웨이건대입구', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '곱창의전당 서초본점', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '옴므', 'output': '쇼핑'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '씨유 관악건영점', 'output': '기타'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '여기어때', 'output': '기타'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': 'OPENAI', 'output': '디지털'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '탐앤탐스 광진화양점', 'output': '카페'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '카츠공방', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': 'OPENAI *CHATGPT SUBSCR',
  'output': '디지털'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '지에스GS25 서초한신점',
  'output

In [46]:
from datasets import Dataset, DatasetDict

# 데이터셋 변환
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(validation_data)

raw_datasets = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

In [48]:
# 데이터 전처리
def preprocess_function(example):
    return {
        "text": f"[INSTRUCTION] {example['instruction']}\n[INPUT] {example['input']}\n[OUTPUT] {example['output']}"
    }

raw_datasets = raw_datasets.map(preprocess_function)

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [59]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [61]:
# 데이터 포맷 변환 함수
def format_llama_prompt(example):
    prompt = f"### 질문: {example['instruction']}\n### 답변: {example['output']}"
    return {"text": prompt}

# 토크나이징 함수
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokenized["labels"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in input_ids]
        for input_ids in tokenized["input_ids"]
    ]
    return tokenized

In [63]:
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [65]:
import wandb

wandb.init(project="CardList")
wandb.run.name = "sft"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [69]:
from transformers import TrainingArguments, Trainer, default_data_collator

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",
    run_name="llama-finetune-1b",
    gradient_checkpointing=True,
    use_cpu=True
)

In [71]:
device

device(type='cpu')

In [73]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

trainer.train()

/var/folders/yz/dp_hy1vd5jj8pzrlts8w9z4m0000gn/T/ipykernel_80102/1334537084.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,1.804900,1.436433
2,0.883800,1.569680
3,0.365400,1.622382


TrainOutput(global_step=189, training_loss=0.994094210326987, metrics={'train_runtime': 4268.7179, 'train_samples_per_second': 0.044, 'train_steps_per_second': 0.044, 'total_flos': 141254104449024.0, 'train_loss': 0.994094210326987, 'epoch': 3.0})